# Step 1

## Task 2

### a)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    matthews_corrcoef, classification_report, make_scorer
)

from sklearn.svm import LinearSVC
from sklearn.ensemble import (
    HistGradientBoostingClassifier,
    VotingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer, MinMaxScaler, MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_validate



import scipy.stats as stats
from scipy.stats import f_oneway, tukey_hsd

import dill
import os

In [2]:
Company = pd.read_csv("Company.csv")
Company.head(5)

,form_end,form_start,q01,q02,q03,q04,q05,q06,q07,q08,...,q65,q66,q67,q68,q69,q70,q71,q72,q73,q74
0,2021-05-26 05:29:10,2021-05-26 05:20:54,1,1,2,1,1,1,1,1,...,"2,3,5","3,4,5","1,3,5",3,3,4,1,1,6,4
1,2021-05-31 09:53:17,2021-05-31 09:44:35,1,2,1,1,1,1,2,1,...,"3,4,5","1,2,7","1,3,4",3,4,5,2,2,3,1
2,2021-05-26 13:01:17,2021-05-26 12:55:24,2,2,1,1,1,2,2,1,...,"3,7","3,7","1,3,4",3,5,5,2,2,4,"3,4"
3,2021-05-27 05:23:09,2021-05-27 05:15:28,2,2,1,1,1,2,2,1,...,"3,6,7","3,4,7","2,5,8",3,3,5,2,1,2,5
4,2021-05-31 10:56:28,2021-05-31 10:51:34,1,1,1,1,1,2,2,1,...,"3,5,7","1,4,7","3,5,7",2,2,4,2,1,6,5


In [3]:
# Company["q61"] = (Company["q61"].astype(int) >= 0).astype(int)
# Company["q62"] = (Company["q62"].astype(int) >= 0).astype(int)

## Task 4

In [4]:
satisfy_monir = Company.copy()
fulfill_monir = Company.copy()

satisfy_monir = satisfy_monir[[column for column in satisfy_monir.columns if column != "q61"] + ["q61"]]  
fulfill_monir = fulfill_monir[[column for column in fulfill_monir.columns if column != "q62"] + ["q62"]]  

In [5]:
satisfy_training_monir, satisfy_test_monir = train_test_split(satisfy_monir, test_size=0.20, random_state=42, shuffle=True)

fulfill_training_monir, fulfill_test_monir = train_test_split(fulfill_monir, test_size=0.20, random_state=42, shuffle=True)

## Task 5

In [6]:
# Class definition obtained from https://stackoverflow.com/questions/59254662/sklearn-columntransformer-with-multilabelbinarizer
class MultiHotEncoder(BaseEstimator, TransformerMixin):
    """Wraps `MultiLabelBinarizer` in a form that can work with `ColumnTransformer`. Note
    that input X has to be a `pandas.DataFrame`.
    """
    def __init__(self):
        self.mlbs = list()
        self.n_columns = 0
        self.categories_ = self.classes_ = list()

    def fit(self, X:pd.DataFrame, y=None):
        for i in range(len(X.columns)): # X can be of multiple columns
            mlb = MultiLabelBinarizer()
            
            col_data = X.iloc[:, i].dropna().apply(
                lambda x: [item.strip() for item in x.split(",")] if isinstance(x, str) else []
            )
            
            mlb.fit(col_data)
            self.mlbs.append(mlb)
            self.classes_.append(mlb.classes_)
            self.n_columns += 1
        return self

    def transform(self, X:pd.DataFrame):
        if self.n_columns == 0:
            raise ValueError('Please fit the transformer first.')
        if self.n_columns != X.shape[1]:
            raise ValueError(f'The fit transformer deals with {self.n_columns} columns '
                             f'while the input has {X.shape[1]}.'
                            )
        result = list()
        
        for i in range(self.n_columns):
            col_data = X.iloc[:, i].dropna().apply(
                lambda x: [item.strip() for item in x.split(",")] if isinstance(x, str) else []
            )
            result.append(self.mlbs[i].transform(col_data))

        result = np.concatenate(result, axis=1)
        return result
    
    def get_feature_names_out(self, input_features):
        feature_names = []
        for i, col in enumerate(input_features):
            feature_names.extend([f"{col}_{cat}" for cat in self.categories_[i]])
        return feature_names

In [7]:
# Not in use anymore

# class DurationFilter(BaseEstimator, TransformerMixin):
#     def __init__(self, min_seconds=0, max_seconds=np.inf):
#         self.min_seconds = min_seconds
#         self.max_seconds = max_seconds

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         start = pd.to_datetime(X["form_start"])
#         end = pd.to_datetime(X["form_end"])
#         duration = (end - start).dt.total_seconds()
#         mask = (duration >= self.min_seconds) & (duration <= self.max_seconds)
#         return X.loc[mask].copy()

In [8]:
# "1":  0.181121,
# "2":  0.037692,
# "3":  0.061610,
# "4": -0.053189,
# "5":  0.120064,
# "6": -0.018297,
# "7": -0.047759,

# "1":  0.180026,
# "2":  0.051272,
# "3":  0.114776,
# "5":  0.077575,
# "6":  0.168781,
# "7":  0.165572,
# "8": -0.211561,

# "1":  0.132330,
# "2":  0.137770,
# "3": -0.054423,
# "4":  0.113744,
# "5":  0.058052,
# "6":  0.104065,
# "7":  0.057903,

# "1": -0.035451,
# "2":  0.174403,
# "3":  0.233034,
# "4":  0.119905,
# "5":  0.068729,
# "6":  0.045352,
# "7": -0.140733,

# "1":  0.091692,
# "2":  0.120436,
# "3":  0.106541,
# "4":  0.057474,
# "5":  0.096895,
# "6":  0.061694,
# "7": -0.064149,
# "8": -0.004011,

# "1": -0.001148,
# "2":  0.035565,
# "3":  0.030810,
# "4":  0.039645,
# "5": -0.045978,
# "6": -0.055425,

# "1":  0.002775,
# "2":  0.016057,
# "3": -0.023039,
# "4": -0.017313,
# "5": -0.000918,

In [9]:
def compute_duration(X):
    if isinstance(X, pd.DataFrame):
        start = pd.to_datetime(X.iloc[:, 0])
        end = pd.to_datetime(X.iloc[:, 1])
        # print((end-start).dt.total_seconds())

    duration = (end - start).dt.total_seconds()
    return duration.values.reshape(-1, 1)

def weight_score_func(df):
    weights = {
        "q63": {"1": 0.181, "2": 0.038, "3": 0.062, "4": -0.053, "5": 0.120, "6": -0.018, "7": -0.048},
        "q64": {"1": 0.180, "2": 0.051, "3": 0.115, "5": 0.078, "6": 0.169, "7": 0.166, "8": -0.212},
        "q65": {"1": 0.132, "2": 0.138, "3": -0.054, "4": 0.114, "5": 0.058, "6": 0.104, "7": 0.058},
        "q66": {"1": -0.035, "2": 0.174, "3": 0.233, "4": 0.120, "5": 0.069, "6": 0.045, "7": -0.141},
        "q67": {"1": 0.092, "2": 0.120, "3": 0.107, "4": 0.057, "5": 0.097, "6": 0.062, "7": -0.064, "8": -0.004},
        "q73": {"1": -0.001, "2": 0.036, "3": 0.031, "4": 0.040, "5": -0.046, "6": -0.055},
        "q74": {"1": 0.003, "2": 0.016, "3": -0.023, "4": -0.017, "5": -0.001}
    }
    
    result = pd.DataFrame(index=df.index)
    
    for col in df.columns:
        all_weights = weights.get(col, {})
        result[col] = df[col].astype(str).apply(
            lambda s: sum(all_weights.get(i.strip(), 0) for i in s.split(",") if i.strip().isdigit())
        )

    return result

def remove_trolls(df, lower=480, upper=1800):
    start = pd.to_datetime(df["form_start"])
    end = pd.to_datetime(df["form_end"])
    duration = (end - start).dt.total_seconds()

    mask = (duration >= lower) & (duration <= upper)
    return df.loc[mask].reset_index(drop=True)  # Reset index to avoid alignment issues later

def normalize_bin(x):
    return (x - 1).astype(int)

In [10]:
satisfy_preprocessing = ColumnTransformer([
    ("duration_pipeline", Pipeline([
        ("compute", FunctionTransformer(compute_duration, validate=False, 
            feature_names_out=lambda self, input_features: np.array(["duration"]))),
        ("scale", MinMaxScaler())
    ]), ["form_start", "form_end"]),
    
    ("binary_select", FunctionTransformer(normalize_bin, feature_names_out="one-to-one"), [f"q{str(i).zfill(2)}" for i in (list(range(1, 61)))] + ["q71", "q72"]),
    ("special_score", MinMaxScaler(), ["q62"]),
    # ("multi_select", MultiHotEncoder(), ["q63", "q64", "q65", "q66", "q67", "q73", "q74"]),
    ("onehot", OneHotEncoder(), ["q68","q69","q70"]),
])

satisfy_pipeline = Pipeline([
    ("preprocessing", satisfy_preprocessing),

])

satisfy_monir_transformed = satisfy_pipeline.fit_transform(remove_trolls(satisfy_monir, 3000000, np.inf))

satisfy_col_names = [i.split("__")[1] for i in satisfy_preprocessing.get_feature_names_out()]

satisfy_monir_transformed = pd.DataFrame(satisfy_monir_transformed, columns=satisfy_col_names)

satisfy_monir_transformed

,duration,q01,q02,q03,q04,q05,q06,q07,q08,q09,...,q62,q68_2,q68_3,q69_3,q69_4,q69_5,q70_4,q70_5,q70_7,q70_8
0,0.005589,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.335,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.660669,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.143520,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.050477,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.000000,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5,0.000000,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [11]:
satisfy_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('duration_pipeline',
                                                  Pipeline(steps=[('compute',
                                                                   FunctionTransformer(feature_names_out=<function <lambda> at 0x137a49760>,
                                                                                       func=<function compute_duration at 0x137a49a80>)),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['form_start', 'form_end']),
                                                 ('binary_select',
                                                  FunctionTransformer(feature_names_out='one-to-one',
                                                                      func=<function normalize_bin at 0x137a4a0c0>),
                                                  ['q01', 'q02', 'q03', 'q04',
                                                   'q05', 'q06', 'q07', 'q08',
                                                   'q09', 'q10', 'q11', 'q12',
                                                   'q13', 'q14', 'q15', 'q16',
                                                   'q17', 'q18', 'q19', 'q20',
                                                   'q21', 'q22', 'q23', 'q24',
                                                   'q25', 'q26', 'q27', 'q28',
                                                   'q29', 'q30', ...]),
                                                 ('special_score',
                                                  MinMaxScaler(), ['q62']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['q68', 'q69', 'q70'])]))])

# Tried & failed:
- PCA
- weighting the multi selects features and adding the weights
- Multi hot encoding the multi selects and weight summing them

In [12]:
fulfill_preprocessing = ColumnTransformer([
    ("duration_pipeline", Pipeline([
        ("compute", FunctionTransformer(compute_duration, validate=False, 
            feature_names_out=lambda self, input_features: np.array(["duration"]))),
        ("scale", MinMaxScaler())
    ]), ["form_start", "form_end"]),
    
    ("binary_select", FunctionTransformer(normalize_bin, feature_names_out="one-to-one"), [f"q{str(i).zfill(2)}" for i in (list(range(1, 61)))] + ["q71", "q72"]),
    ("special_score", MinMaxScaler(), ["q61"]),
    ("multi_select", MultiHotEncoder(), ["q63", "q64", "q65", "q66", "q67", "q73", "q74"]),
    ("onehot", OneHotEncoder(), ["q68","q69","q70"]),
])

fulfill_pipeline = Pipeline([
    ("preprocessing", fulfill_preprocessing)
])

fulfill_monir_transformed = fulfill_pipeline.fit_transform(fulfill_monir)

fulfill_col_names = [i.split("__")[1] for i in fulfill_preprocessing.get_feature_names_out()]

fulfill_monir_transformed = pd.DataFrame(fulfill_monir_transformed, columns=fulfill_col_names)

fulfill_monir_transformed.tail()

,duration,q01,q02,q03,q04,q05,q06,q07,q08,q09,...,q69_6,q70_1,q70_2,q70_3,q70_4,q70_5,q70_6,q70_7,q70_8,q70_9
29794,0.000028,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
29795,0.000024,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
29796,0.000014,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
29797,0.000028,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
29798,0.000075,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
fulfill_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('duration_pipeline',
                                                  Pipeline(steps=[('compute',
                                                                   FunctionTransformer(feature_names_out=<function <lambda> at 0x137a4b240>,
                                                                                       func=<function compute_duration at 0x137a49a80>)),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['form_start', 'form_end']),
                                                 ('binary_select',
                                                  FunctionTransformer(feature_names_out='one-to-one...
                                                  ['q01', 'q02', 'q03', 'q04',
                                                   'q05', 'q06', 'q07', 'q08',
                                                   'q09', 'q10', 'q11', 'q12',
                                                   'q13', 'q14', 'q15', 'q16',
                                                   'q17', 'q18', 'q19', 'q20',
                                                   'q21', 'q22', 'q23', 'q24',
                                                   'q25', 'q26', 'q27', 'q28',
                                                   'q29', 'q30', ...]),
                                                 ('special_score',
                                                  MinMaxScaler(), ['q61']),
                                                 ('multi_select',
                                                  MultiHotEncoder(),
                                                  ['q63', 'q64', 'q65', 'q66',
                                                   'q67', 'q73', 'q74']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['q68', 'q69', 'q70'])]))])

# Step 2

In [14]:
satisfy_training_monir_X = remove_trolls(satisfy_training_monir).drop(columns=["q61"])
satisfy_training_monir_y = remove_trolls(satisfy_training_monir)["q61"]

fulfill_training_monir_X = remove_trolls(fulfill_training_monir).drop(columns=["q62"])
fulfill_training_monir_y = remove_trolls(fulfill_training_monir)["q62"]

## Task 6

### Decision Stump

#### Satisfy

In [15]:
satisfy_stump_pipeline = make_pipeline(
    satisfy_preprocessing,
    DecisionTreeClassifier(max_depth=1)
)

# Fitting the input model to make predictions based on the target feature
satisfy_stump_pipeline.fit(satisfy_training_monir_X, satisfy_training_monir_y)

# Getting predicted values
y_pred = satisfy_stump_pipeline.predict(satisfy_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_training_monir_y, y_pred)
class_rep = classification_report(satisfy_training_monir_y, y_pred)
cm = confusion_matrix(satisfy_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.637842824792418

Classification Report:
               precision    recall  f1-score   support

        -100       0.00      0.00      0.00       227
         -33       0.00      0.00      0.00      1026
          33       0.68      0.64      0.66      6373
         100       0.59      0.82      0.69      4297

    accuracy                           0.64     11923
   macro avg       0.32      0.37      0.34     11923
weighted avg       0.58      0.64      0.60     11923

Confusion Matrix:
 [[   0    0  205   22]
 [   0    0  915  111]
 [   0    0 4074 2299]
 [   0    0  766 3531]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [16]:
# Separating the input features and target feature
satisfy_test_monir_X = satisfy_test_monir.drop(columns=["q61"])
satisfy_test_monir_y = satisfy_test_monir["q61"]

# Getting predicted values
y_pred = satisfy_stump_pipeline.predict(satisfy_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_test_monir_y, y_pred)
class_rep = classification_report(satisfy_test_monir_y, y_pred)
cm = confusion_matrix(satisfy_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.6288590604026846

Classification Report:
               precision    recall  f1-score   support

        -100       0.00      0.00      0.00       125
         -33       0.00      0.00      0.00       594
          33       0.67      0.64      0.66      3186
         100       0.58      0.83      0.68      2055

    accuracy                           0.63      5960
   macro avg       0.31      0.37      0.34      5960
weighted avg       0.56      0.63      0.59      5960

Confusion Matrix:
 [[   0    0  107   18]
 [   0    0  530   64]
 [   0    0 2042 1144]
 [   0    0  349 1706]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

#### Fulfill

In [17]:
fulfill_stump_pipeline = make_pipeline(
    fulfill_preprocessing,
    DecisionTreeClassifier(max_depth=1)
)

# Fitting the input model to make predictions based on the target feature
fulfill_stump_pipeline.fit(fulfill_training_monir_X, fulfill_training_monir_y)

# Getting predicted values
y_pred = fulfill_stump_pipeline.predict(fulfill_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_training_monir_y, y_pred)
class_rep = classification_report(fulfill_training_monir_y, y_pred)
cm = confusion_matrix(fulfill_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.5943135117000755

Classification Report:
               precision    recall  f1-score   support

        -100       0.00      0.00      0.00       537
         -33       0.00      0.00      0.00      1210
          33       0.47      0.84      0.60      4213
         100       0.82      0.59      0.69      5963

    accuracy                           0.59     11923
   macro avg       0.32      0.36      0.32     11923
weighted avg       0.58      0.59      0.56     11923

Confusion Matrix:
 [[   0    0  494   43]
 [   0    0 1145   65]
 [   0    0 3555  658]
 [   0    0 2432 3531]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [18]:
# Separating the input features and target feature
fulfill_test_monir_X = fulfill_test_monir.drop(columns=["q62"])
fulfill_test_monir_y = fulfill_test_monir["q62"]

# Getting predicted values
y_pred = fulfill_stump_pipeline.predict(fulfill_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_test_monir_y, y_pred)
class_rep = classification_report(fulfill_test_monir_y, y_pred)
cm = confusion_matrix(fulfill_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.5840604026845637

Classification Report:
               precision    recall  f1-score   support

        -100       0.00      0.00      0.00       298
         -33       0.00      0.00      0.00       654
          33       0.45      0.86      0.59      2076
         100       0.83      0.58      0.68      2932

    accuracy                           0.58      5960
   macro avg       0.32      0.36      0.32      5960
weighted avg       0.57      0.58      0.54      5960

Confusion Matrix:
 [[   0    0  279   19]
 [   0    0  625   29]
 [   0    0 1775  301]
 [   0    0 1226 1706]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

### Decision Tree

#### Satisfy

In [19]:
satisfy_tree_pipeline = make_pipeline(
    satisfy_preprocessing,
    DecisionTreeClassifier(max_depth=15)
)

# Fitting the input model to make predictions based on the target feature
satisfy_tree_pipeline.fit(satisfy_training_monir_X, satisfy_training_monir_y)

# Getting predicted values
y_pred = satisfy_tree_pipeline.predict(satisfy_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_training_monir_y, y_pred)
class_rep = classification_report(satisfy_training_monir_y, y_pred)
cm = confusion_matrix(satisfy_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.9448964186865721

Classification Report:
               precision    recall  f1-score   support

        -100       0.99      0.92      0.95       227
         -33       0.95      0.90      0.93      1026
          33       0.95      0.96      0.95      6373
         100       0.94      0.94      0.94      4297

    accuracy                           0.94     11923
   macro avg       0.96      0.93      0.94     11923
weighted avg       0.94      0.94      0.94     11923

Confusion Matrix:
 [[ 209    3   13    2]
 [   0  928   82   16]
 [   2   32 6088  251]
 [   1   12  243 4041]]


In [20]:
# Separating the input features and target feature
satisfy_test_monir_X = satisfy_test_monir.drop(columns=["q61"])
satisfy_test_monir_y = satisfy_test_monir["q61"]

# Getting predicted values
y_pred = satisfy_tree_pipeline.predict(satisfy_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_test_monir_y, y_pred)
class_rep = classification_report(satisfy_test_monir_y, y_pred)
cm = confusion_matrix(satisfy_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.5156040268456376

Classification Report:
               precision    recall  f1-score   support

        -100       0.12      0.10      0.11       125
         -33       0.24      0.27      0.26       594
          33       0.60      0.55      0.58      3186
         100       0.51      0.55      0.53      2055

    accuracy                           0.52      5960
   macro avg       0.37      0.37      0.37      5960
weighted avg       0.52      0.52      0.52      5960

Confusion Matrix:
 [[  13   44   38   30]
 [  38  163  303   90]
 [  43  396 1764  983]
 [  11   80  831 1133]]


#### Fulfill

In [21]:
fulfill_tree_pipeline = make_pipeline(
    fulfill_preprocessing,
    DecisionTreeClassifier(max_depth=15)
)

# Fitting the input model to make predictions based on the target feature
fulfill_tree_pipeline.fit(fulfill_training_monir_X, fulfill_training_monir_y)

# Getting predicted values
y_pred = fulfill_tree_pipeline.predict(fulfill_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_training_monir_y, y_pred)
class_rep = classification_report(fulfill_training_monir_y, y_pred)
cm = confusion_matrix(fulfill_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.912018787217982

Classification Report:
               precision    recall  f1-score   support

        -100       0.95      0.82      0.88       537
         -33       0.95      0.81      0.87      1210
          33       0.86      0.93      0.89      4213
         100       0.94      0.93      0.93      5963

    accuracy                           0.91     11923
   macro avg       0.92      0.87      0.90     11923
weighted avg       0.91      0.91      0.91     11923

Confusion Matrix:
 [[ 441    7   62   27]
 [   6  984  166   54]
 [   8   30 3919  256]
 [  11   20  402 5530]]


In [22]:
# Separating the input features and target feature
fulfill_test_monir_X = fulfill_test_monir.drop(columns=["q62"])
fulfill_test_monir_y = fulfill_test_monir["q62"]

# Getting predicted values
y_pred = fulfill_tree_pipeline.predict(fulfill_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_test_monir_y, y_pred)
class_rep = classification_report(fulfill_test_monir_y, y_pred)
cm = confusion_matrix(fulfill_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.500503355704698

Classification Report:
               precision    recall  f1-score   support

        -100       0.25      0.24      0.25       298
         -33       0.27      0.24      0.25       654
          33       0.43      0.49      0.46      2076
         100       0.63      0.59      0.61      2932

    accuracy                           0.50      5960
   macro avg       0.40      0.39      0.39      5960
weighted avg       0.51      0.50      0.50      5960

Confusion Matrix:
 [[  73   68   94   63]
 [  61  156  272  165]
 [  80  204 1027  765]
 [  75  156  974 1727]]


### Random Forest

#### Satisfy

In [23]:
satisfy_RF_pipeline = make_pipeline(
    satisfy_preprocessing,
    RandomForestClassifier(
            n_estimators=300,             
            max_depth=20,              
            min_samples_split=5,         
            min_samples_leaf=4,           
            max_features="sqrt",      
            class_weight="balanced",     
            bootstrap=True,              
            random_state=42,
    )
)

# Fitting the input model to make predictions based on the target feature
satisfy_RF_pipeline.fit(satisfy_training_monir_X, satisfy_training_monir_y)

# Getting predicted values
y_pred = satisfy_RF_pipeline.predict(satisfy_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_training_monir_y, y_pred)
class_rep = classification_report(satisfy_training_monir_y, y_pred)
cm = confusion_matrix(satisfy_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.8829153736475719

Classification Report:
               precision    recall  f1-score   support

        -100       0.97      1.00      0.98       227
         -33       0.71      0.99      0.83      1026
          33       0.94      0.84      0.89      6373
         100       0.86      0.91      0.88      4297

    accuracy                           0.88     11923
   macro avg       0.87      0.94      0.90     11923
weighted avg       0.89      0.88      0.88     11923

Confusion Matrix:
 [[ 227    0    0    0]
 [   0 1018    0    8]
 [   6  335 5383  649]
 [   1   71  326 3899]]


In [24]:
# Separating the input features and target feature
satisfy_test_monir_X = satisfy_test_monir.drop(columns=["q61"])
satisfy_test_monir_y = satisfy_test_monir["q61"]

# Getting predicted values
y_pred = satisfy_RF_pipeline.predict(satisfy_test_monir_X)

accuracy = accuracy_score(satisfy_test_monir_y, y_pred)
f_1 = f1_score(satisfy_test_monir_y, y_pred, average='macro')
precision = precision_score(satisfy_test_monir_y, y_pred, average='macro', zero_division=1)
recall = recall_score(satisfy_test_monir_y, y_pred, average='macro', zero_division=1)
mcc = matthews_corrcoef(satisfy_test_monir_y, y_pred)

# Printing metrics
print("\nRandomForestClassifier Evaluation on Test Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f_1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"MCC: {mcc:.4f}")




RandomForestClassifier Evaluation on Test Set:
Accuracy: 0.6376
F1 Score: 0.4969
Precision: 0.5521
Recall: 0.5055
MCC: 0.4021


#### Fulfill

In [25]:
fulfill_RF_pipeline = make_pipeline(
    fulfill_preprocessing,
        RandomForestClassifier(
            n_estimators=300,             
            max_depth=20,              
            min_samples_split=5,         
            min_samples_leaf=4,           
            max_features="sqrt",      
            class_weight="balanced",     
            bootstrap=True,              
            random_state=42,
    )
)

# Fitting the input model to make predictions based on the target feature
fulfill_RF_pipeline.fit(fulfill_training_monir_X, fulfill_training_monir_y)

# Getting predicted values
y_pred = fulfill_RF_pipeline.predict(fulfill_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_training_monir_y, y_pred)
class_rep = classification_report(fulfill_training_monir_y, y_pred)
cm = confusion_matrix(fulfill_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.9485028935670553

Classification Report:
               precision    recall  f1-score   support

        -100       0.95      1.00      0.98       537
         -33       0.89      0.99      0.94      1210
          33       0.95      0.93      0.94      4213
         100       0.96      0.95      0.95      5963

    accuracy                           0.95     11923
   macro avg       0.94      0.97      0.95     11923
weighted avg       0.95      0.95      0.95     11923

Confusion Matrix:
 [[ 537    0    0    0]
 [   0 1202    0    8]
 [   9   69 3907  228]
 [  17   77  206 5663]]


In [26]:
# Separating the input features and target feature
fulfill_test_monir_X = fulfill_test_monir.drop(columns=["q62"])
fulfill_test_monir_y = fulfill_test_monir["q62"]

# Getting predicted values
y_pred = fulfill_RF_pipeline.predict(fulfill_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_test_monir_y, y_pred)
class_rep = classification_report(fulfill_test_monir_y, y_pred)
cm = confusion_matrix(fulfill_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.616275167785235

Classification Report:
               precision    recall  f1-score   support

        -100       0.52      0.19      0.28       298
         -33       0.45      0.35      0.39       654
          33       0.52      0.69      0.59      2076
         100       0.76      0.67      0.71      2932

    accuracy                           0.62      5960
   macro avg       0.56      0.47      0.49      5960
weighted avg       0.63      0.62      0.61      5960

Confusion Matrix:
 [[  56  118   96   28]
 [  30  227  331   66]
 [  10  113 1426  527]
 [  12   51  905 1964]]


#### Satisfy

In [27]:
satisfy_Ada_pipeline = make_pipeline(
    satisfy_preprocessing,
    AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=8, class_weight="balanced"),
        n_estimators=50,
        learning_rate=0.5
    )
)

# Fitting the input model to make predictions based on the target feature
satisfy_Ada_pipeline.fit(satisfy_training_monir_X, satisfy_training_monir_y)

# Getting predicted values
y_pred = satisfy_Ada_pipeline.predict(satisfy_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_training_monir_y, y_pred)
class_rep = classification_report(satisfy_training_monir_y, y_pred)
cm = confusion_matrix(satisfy_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.8757862953954542

Classification Report:
               precision    recall  f1-score   support

        -100       1.00      1.00      1.00       227
         -33       0.97      0.99      0.98      1026
          33       0.90      0.86      0.88      6373
         100       0.81      0.87      0.84      4297

    accuracy                           0.88     11923
   macro avg       0.92      0.93      0.93     11923
weighted avg       0.88      0.88      0.88     11923

Confusion Matrix:
 [[ 227    0    0    0]
 [   0 1018    8    0]
 [   0   28 5466  879]
 [   0    0  566 3731]]


In [28]:
# Separating the input features and target feature
satisfy_test_monir_X = satisfy_test_monir.drop(columns=["q61"])
satisfy_test_monir_y = satisfy_test_monir["q61"]

# Getting predicted values
y_pred = satisfy_Ada_pipeline.predict(satisfy_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_test_monir_y, y_pred)
class_rep = classification_report(satisfy_test_monir_y, y_pred)
cm = confusion_matrix(satisfy_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.5879194630872483

Classification Report:
               precision    recall  f1-score   support

        -100       0.25      0.17      0.20       125
         -33       0.42      0.37      0.39       594
          33       0.65      0.62      0.64      3186
         100       0.55      0.63      0.59      2055

    accuracy                           0.59      5960
   macro avg       0.47      0.45      0.45      5960
weighted avg       0.59      0.59      0.59      5960

Confusion Matrix:
 [[  21   47   41   16]
 [  40  221  276   57]
 [  18  231 1973  964]
 [   4   31  731 1289]]


#### Fulfill

In [29]:
fulfill_Ada_pipeline = make_pipeline(
    fulfill_preprocessing,
    AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=9, class_weight="balanced"),
        n_estimators=50,
        learning_rate=0.5
    ))

# Fitting the input model to make predictions based on the target feature
fulfill_Ada_pipeline.fit(fulfill_training_monir_X, fulfill_training_monir_y)

# Getting predicted values
y_pred = fulfill_Ada_pipeline.predict(fulfill_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_training_monir_y, y_pred)
class_rep = classification_report(fulfill_training_monir_y, y_pred)
cm = confusion_matrix(fulfill_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.976599849031284

Classification Report:
               precision    recall  f1-score   support

        -100       1.00      1.00      1.00       537
         -33       1.00      1.00      1.00      1210
          33       0.96      0.98      0.97      4213
         100       0.98      0.97      0.98      5963

    accuracy                           0.98     11923
   macro avg       0.98      0.99      0.99     11923
weighted avg       0.98      0.98      0.98     11923

Confusion Matrix:
 [[ 537    0    0    0]
 [   0 1209    1    0]
 [   0    1 4114   98]
 [   0    1  178 5784]]


In [30]:
# Separating the input features and target feature
fulfill_test_monir_X = fulfill_test_monir.drop(columns=["q62"])
fulfill_test_monir_y = fulfill_test_monir["q62"]

# Getting predicted values
y_pred = fulfill_Ada_pipeline.predict(fulfill_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_test_monir_y, y_pred)
class_rep = classification_report(fulfill_test_monir_y, y_pred)
cm = confusion_matrix(fulfill_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.5808724832214766

Classification Report:
               precision    recall  f1-score   support

        -100       0.47      0.22      0.30       298
         -33       0.42      0.36      0.39       654
          33       0.49      0.58      0.53      2076
         100       0.70      0.67      0.68      2932

    accuracy                           0.58      5960
   macro avg       0.52      0.46      0.48      5960
weighted avg       0.58      0.58      0.58      5960

Confusion Matrix:
 [[  67  102   82   47]
 [  46  235  286   87]
 [  12  157 1208  699]
 [  18   72  890 1952]]


### SVC

#### Satisfy

In [31]:
satisfy_svc_pipeline = make_pipeline(
    satisfy_preprocessing,
    LinearSVC(C=10, max_iter=5000)
)

# Fitting the input model to make predictions based on the target feature
satisfy_svc_pipeline.fit(satisfy_training_monir_X, satisfy_training_monir_y)

# Getting predicted values
y_pred = satisfy_svc_pipeline.predict(satisfy_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_training_monir_y, y_pred)
class_rep = classification_report(satisfy_training_monir_y, y_pred)
cm = confusion_matrix(satisfy_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.6469848192568984

Classification Report:
               precision    recall  f1-score   support

        -100       0.00      0.00      0.00       227
         -33       0.50      0.00      0.01      1026
          33       0.64      0.78      0.71      6373
         100       0.65      0.63      0.64      4297

    accuracy                           0.65     11923
   macro avg       0.45      0.35      0.34     11923
weighted avg       0.62      0.65      0.61     11923

Confusion Matrix:
 [[   0    1  218    8]
 [   0    3  979   44]
 [   0    1 4987 1385]
 [   0    1 1572 2724]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [32]:
# Separating the input features and target feature
satisfy_test_monir_X = satisfy_test_monir.drop(columns=["q61"])
satisfy_test_monir_y = satisfy_test_monir["q61"]

# Getting predicted values
y_pred = satisfy_svc_pipeline.predict(satisfy_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_test_monir_y, y_pred)
class_rep = classification_report(satisfy_test_monir_y, y_pred)
cm = confusion_matrix(satisfy_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.6288590604026846

Classification Report:
               precision    recall  f1-score   support

        -100       0.00      0.00      0.00       125
         -33       0.00      0.00      0.00       594
          33       0.63      0.77      0.69      3186
         100       0.63      0.63      0.63      2055

    accuracy                           0.63      5960
   macro avg       0.31      0.35      0.33      5960
weighted avg       0.55      0.63      0.59      5960

Confusion Matrix:
 [[   0    0  114   11]
 [   0    0  560   34]
 [   0    1 2461  724]
 [   0    0  768 1287]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

#### Fulfill

In [33]:
fulfill_svc_pipeline = make_pipeline(
    fulfill_preprocessing,
    LinearSVC(C=10, max_iter=5000)
)

# Fitting the input model to make predictions based on the target feature
fulfill_svc_pipeline.fit(fulfill_training_monir_X, fulfill_training_monir_y)

# Getting predicted values
y_pred = fulfill_svc_pipeline.predict(fulfill_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_training_monir_y, y_pred)
class_rep = classification_report(fulfill_training_monir_y, y_pred)
cm = confusion_matrix(fulfill_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.5984232156336492

Classification Report:
               precision    recall  f1-score   support

        -100       0.53      0.01      0.03       537
         -33       0.43      0.05      0.09      1210
          33       0.49      0.50      0.49      4213
         100       0.66      0.83      0.74      5963

    accuracy                           0.60     11923
   macro avg       0.53      0.35      0.34     11923
weighted avg       0.57      0.60      0.55     11923

Confusion Matrix:
 [[   8   49  357  123]
 [   2   64  836  308]
 [   1   17 2086 2109]
 [   4   19  963 4977]]


In [34]:
# Separating the input features and target feature
fulfill_test_monir_X = fulfill_test_monir.drop(columns=["q62"])
fulfill_test_monir_y = fulfill_test_monir["q62"]

# Getting predicted values
y_pred = fulfill_svc_pipeline.predict(fulfill_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_test_monir_y, y_pred)
class_rep = classification_report(fulfill_test_monir_y, y_pred)
cm = confusion_matrix(fulfill_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.5689597315436241

Classification Report:
               precision    recall  f1-score   support

        -100       0.36      0.01      0.03       298
         -33       0.37      0.05      0.09       654
          33       0.45      0.52      0.48      2076
         100       0.66      0.78      0.71      2932

    accuracy                           0.57      5960
   macro avg       0.46      0.34      0.33      5960
weighted avg       0.54      0.57      0.53      5960

Confusion Matrix:
 [[   4   25  209   60]
 [   3   32  476  143]
 [   3   16 1075  982]
 [   1   14  637 2280]]


### Categorical Native Bayes

#### Satisfy

In [35]:
satisfy_NB_pipeline = make_pipeline(
    satisfy_preprocessing,
    CategoricalNB()
)

# Fitting the input model to make predictions based on the target feature
satisfy_NB_pipeline.fit(satisfy_training_monir_X, satisfy_training_monir_y)

# Getting predicted values
y_pred = satisfy_NB_pipeline.predict(satisfy_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_training_monir_y, y_pred)
class_rep = classification_report(satisfy_training_monir_y, y_pred)
cm = confusion_matrix(satisfy_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

Accuracy: 0.6022813050406777

Classification Report:
               precision    recall  f1-score   support

        -100       0.13      0.08      0.10       227
         -33       0.27      0.20      0.23      1026
          33       0.65      0.66      0.65      6373
         100       0.61      0.65      0.63      4297

    accuracy                           0.60     11923
   macro avg       0.41      0.40      0.40     11923
weighted avg       0.59      0.60      0.60     11923

Confusion Matrix:
 [[  18   49  146   14]
 [  33  207  703   83]
 [  65  455 4182 1671]
 [  19   54 1450 2774]]


In [36]:
# Separating the input features and target feature
satisfy_test_monir_X = satisfy_test_monir.drop(columns=["q61"])
satisfy_test_monir_y = satisfy_test_monir["q61"]

# Getting predicted values
y_pred = satisfy_NB_pipeline.predict(satisfy_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_test_monir_y, y_pred)
class_rep = classification_report(satisfy_test_monir_y, y_pred)
cm = confusion_matrix(satisfy_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

IndexError: index 59 is out of bounds for axis 1 with size 1

#### Fulfill

In [ ]:
fulfill_NB_pipeline = make_pipeline(
    fulfill_preprocessing,
    CategoricalNB()
)

# Fitting the input model to make predictions based on the target feature
fulfill_NB_pipeline.fit(fulfill_training_monir_X, fulfill_training_monir_y)

# Getting predicted values
y_pred = fulfill_NB_pipeline.predict(fulfill_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_training_monir_y, y_pred)
class_rep = classification_report(fulfill_training_monir_y, y_pred)
cm = confusion_matrix(fulfill_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

In [ ]:
# Separating the input features and target feature
fulfill_test_monir_X = fulfill_test_monir.drop(columns=["q62"])
fulfill_test_monir_y = fulfill_test_monir["q62"]

# Getting predicted values
y_pred = fulfill_NB_pipeline.predict(fulfill_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_test_monir_y, y_pred)
class_rep = classification_report(fulfill_test_monir_y, y_pred)
cm = confusion_matrix(fulfill_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

### Logistic Regression

#### Satisfy

In [ ]:
satisfy_LR_pipeline = Pipeline([
    ("satis", satisfy_pipeline),
    ("lg" , LogisticRegression(C=10, max_iter=200))
])
# Fitting the input model to make predictions based on the target feature
satisfy_LR_pipeline.fit(satisfy_training_monir_X, satisfy_training_monir_y)

# Getting predicted values
y_pred = satisfy_LR_pipeline.predict(satisfy_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_training_monir_y, y_pred)
class_rep = classification_report(satisfy_training_monir_y, y_pred)
cm = confusion_matrix(satisfy_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

In [ ]:
# Separating the input features and target feature
satisfy_test_monir_X = satisfy_test_monir.drop(columns=["q61"])
satisfy_test_monir_y = satisfy_test_monir["q61"]

# Getting predicted values
y_pred = satisfy_LR_pipeline.predict(satisfy_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(satisfy_test_monir_y, y_pred)
class_rep = classification_report(satisfy_test_monir_y, y_pred)
cm = confusion_matrix(satisfy_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

#### Fulfull

In [ ]:
fulfill_LR_pipeline = make_pipeline(
    fulfill_preprocessing,
    LogisticRegression(C=10, max_iter=200)
)

# Fitting the input model to make predictions based on the target feature
fulfill_LR_pipeline.fit(fulfill_training_monir_X, fulfill_training_monir_y)

# Getting predicted values
y_pred = fulfill_LR_pipeline.predict(fulfill_training_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_training_monir_y, y_pred)
class_rep = classification_report(fulfill_training_monir_y, y_pred)
cm = confusion_matrix(fulfill_training_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

In [ ]:
# Separating the input features and target feature
fulfill_test_monir_X = fulfill_test_monir.drop(columns=["q62"])
fulfill_test_monir_y = fulfill_test_monir["q62"]

# Getting predicted values
y_pred = fulfill_LR_pipeline.predict(fulfill_test_monir_X)

# Calculating the accuracy, classification report, and confusion matric
accuracy = accuracy_score(fulfill_test_monir_y, y_pred)
class_rep = classification_report(fulfill_test_monir_y, y_pred)
cm = confusion_matrix(fulfill_test_monir_y, y_pred)

# Printing metrics
print("Accuracy:", accuracy)
print("\nClassification Report:\n", class_rep)
print("Confusion Matrix:\n", cm)

In [ ]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1)*(r - 1)) / (n - 1))    
    rcorr = r - ((r - 1)**2) / (n - 1)
    kcorr = k - ((k - 1)**2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

# Example
for i in range(1,75):
    if i != 61:
        col = f"q{i:02}"
        print(f"{col}: {cramers_v(Company["q61"], Company[col])}")

### XGBoost

#### Satisfy

In [ ]:
# Define pipeline with XGBoost
satisfy_XGB_pipeline = make_pipeline(
    satisfy_pipeline,
    XGBClassifier(
        objective='multi:softprob',
        num_class=4,
        learning_rate=0.1,
        n_estimators= 100,
        random_state=42
    )
)

satisfy_training_monir_trollfree = remove_trolls(satisfy_training_monir, lower = 480, upper=1800)
print(len(satisfy_training_monir_trollfree))
X_train = satisfy_training_monir_trollfree.drop(columns=["q61"])
y_train = satisfy_training_monir_trollfree["q61"]

satisfy_target_encoder = LabelEncoder()
satisfy_XGB_pipeline.fit(X_train, satisfy_target_encoder.fit_transform(y_train))

y_pred = satisfy_XGB_pipeline.predict(X_train)
y_pred = satisfy_target_encoder.inverse_transform(y_pred)

print("\nFinal Evaluation on Training Set:")
print(f"Accuracy: {accuracy_score(y_train, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_train, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_train, y_pred, average='macro', zero_division=1):.4f}")
print(f"Recall: {recall_score(y_train, y_pred, average='macro', zero_division=1):.4f}")
print(f"MCC: {matthews_corrcoef(y_train, y_pred):.4f}")

In [ ]:
# Final test set evaluation
X_test = satisfy_test_monir.drop(columns=["q61"])
y_test = satisfy_test_monir["q61"]

y_pred = satisfy_XGB_pipeline.predict(X_test)
y_pred = satisfy_target_encoder.inverse_transform(y_pred)

print("\nXGBClassifier Evaluation on Test Set:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro', zero_division=1):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro', zero_division=1):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred):.4f}")

In [ ]:
fulfill_XGB_pipeline = make_pipeline(
    fulfill_pipeline,
    XGBClassifier(
        objective='multi:softprob',
        num_class=4,
        learning_rate=0.1,
        n_estimators= 100,
        random_state=42
    )
)

fulfill_training_monir_trollfree = remove_trolls(fulfill_training_monir, lower = 480, upper=1800)
print(len(fulfill_training_monir_trollfree))
X_train = fulfill_training_monir_trollfree.drop(columns=["q62"])
y_train = fulfill_training_monir_trollfree["q62"]

fulfill_target_encoder = LabelEncoder()
fulfill_XGB_pipeline.fit(X_train, fulfill_target_encoder.fit_transform(y_train))

y_pred = fulfill_XGB_pipeline.predict(X_train)
y_pred = fulfill_target_encoder.inverse_transform(y_pred)

print("\nFinal Evaluation on Test Set:")
print(f"Accuracy: {accuracy_score(y_train, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_train, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_train, y_pred, average='macro', zero_division=1):.4f}")
print(f"Recall: {recall_score(y_train, y_pred, average='macro', zero_division=1):.4f}")
print(f"MCC: {matthews_corrcoef(y_train, y_pred):.4f}")

In [ ]:
# Final test set evaluation
X_test = fulfill_test_monir.drop(columns=["q62"])
y_test = fulfill_test_monir["q62"]

y_pred = fulfill_XGB_pipeline.predict(X_test)
y_pred = fulfill_target_encoder.inverse_transform(y_pred)

print("\nFinal Evaluation on Test Set:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro', zero_division=1):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro', zero_division=1):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred):.4f}")

# Step 4

## Mali Models

In [ ]:
os.makedirs("saved_models", exist_ok=True)

X_train = satisfy_training_monir.drop(columns=["q61"])
y_train = satisfy_training_monir["q61"]



def evaluate_model(name, classifier):
    pipeline = Pipeline([
        ("preprocessing", satisfy_preprocessing),
        ("classifier", classifier)
    ])
    
    scoring = {
        'accuracy': 'accuracy',
        'mcc': make_scorer(matthews_corrcoef),
        'f1': 'f1_macro',
        'precision': make_scorer(precision_score, zero_division=1, average='macro'),
        'recall': make_scorer(recall_score, zero_division=1, average='macro')
    }

    cv_results = cross_validate(pipeline, X_train, y_train, cv=4, scoring=scoring, return_train_score=True, n_jobs=-1)

    print(f"\n{name}")
    print(f"{'Metric':<20}{'Train Avg':>12}{'Test Avg':>12}")
    print("-" * 60)
    for metric in scoring:
        train_scores = cv_results[f"train_{metric}"]
        test_scores = cv_results[f"test_{metric}"]
        print(f"{metric:<20}{np.mean(train_scores):>12.4f}{np.mean(test_scores):>12.4f}")
    
    # Return both: scores and the pipeline
    return {metric: cv_results[f"test_{metric}"] for metric in scoring}, pipeline


# Define models
base_models = [
    ("Voting", VotingClassifier(estimators=[
        ("hgb", HistGradientBoostingClassifier(
            learning_rate=0.03, max_iter=300, max_leaf_nodes=31, min_samples_leaf=20,
            max_depth=7, l2_regularization=0.2, early_stopping=True,
            n_iter_no_change=15, random_state=42, class_weight="balanced"
        )),
        ("RandomForest", RandomForestClassifier(
            n_estimators=300,             
            max_depth=20,              
            min_samples_split=5,         
            min_samples_leaf=4,           
            max_features="sqrt",      
            class_weight="balanced",     
            bootstrap=True,              
            random_state=42,
            n_jobs=-1                    
        )),
    ], voting='hard')),
]

cv_scores = {}

for name, clf in base_models:
    model_scores, pipeline = evaluate_model(name, clf)  

    for metric, scores in model_scores.items():
        if metric not in cv_scores:
            cv_scores[metric] = {}
        cv_scores[metric][name] = scores

    # pipeline.fit(X_train, y_train)

    # filename = f"saved_models/{name.replace(' ', '_').lower()}.dill"
    # with open(filename, "wb") as f:
    #     dill.dump(pipeline, f)

    


### Time Hypothesis Training

In [ ]:
len(satisfy_training_monir)

In [ ]:
satisfy_mali_model = Pipeline([
    ("preprocessing", satisfy_pipeline),
    ("Voting", VotingClassifier(estimators=[
        ("hgb", HistGradientBoostingClassifier(
            learning_rate=0.02883701168472723, max_iter=312, max_leaf_nodes=37, min_samples_leaf=15,
            max_depth=7, l2_regularization=0.3398231196727128, early_stopping=True,
            n_iter_no_change=15, random_state=42, class_weight="balanced"
        )),
        ("RandomForest", RandomForestClassifier(
            n_estimators=379,             
            max_depth=18,              
            min_samples_split=5,         
            min_samples_leaf=3,           
            max_features="sqrt",      
            class_weight="balanced",     
            bootstrap=False,              
            random_state=42,
        )),
    ], voting='hard')),
])

### Stats for HGBoost tuning:


learning_rate=0.02883701168472723, max_iter=312, max_leaf_nodes=37, min_samples_leaf=15, max_depth=7, l2_regularization=0.3398231196727128, early_stopping=True, n_iter_no_change=15, random_state=42, class_weight="balanced"
- TRAINING
    - Final Evaluation on Training Set:
    - Accuracy: 0.8525
    - F1 Score: 0.7621
    - Precision: 0.7145
    - Recall: 0.8963
    - MCC: 0.7614
- TEST
    - Final Evaluation on Test Set:
    - Accuracy: 0.6332
    - F1 Score: 0.5231
    - Precision: 0.4991
    - Recall: 0.5846
    - MCC: 0.3988

#### <hr/>

## Satisfy

### Time Hypothesis Training

In [ ]:
# Final training set evaluation
satisfy_training_monir_trollfree = remove_trolls(satisfy_training_monir, lower = 480, upper=1800)
print(len(satisfy_training_monir_trollfree))
X_train = satisfy_training_monir_trollfree.drop(columns=["q61"])
y_train = satisfy_training_monir_trollfree["q61"]

satisfy_mali_model.fit(X_train, y_train)

y_pred = satisfy_mali_model.predict(X_train)

len(y_pred)

print("\nVotingClassifier Evaluation on Training Set:")
print(f"Accuracy: {accuracy_score(y_train, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_train, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_train, y_pred, average='macro', zero_division=1):.4f}")
print(f"Recall: {recall_score(y_train, y_pred, average='macro', zero_division=1):.4f}")
print(f"MCC: {matthews_corrcoef(y_train, y_pred):.4f}")

11923

VotingClassifier Evaluation on Training Set:
Accuracy: 0.8380
F1 Score: 0.7509
Precision: 0.7017
Recall: 0.8878
MCC: 0.7391


In [ ]:
with open('satisfy_RubberCorn.dill', 'wb') as file:
    dill.dump(satisfy_mali_model, file)

#### Normal
Final Evaluation on Training Set:   
Accuracy: 0.8109    
F1 Score: 0.7385    
Precision: 0.6879   
Recall: 0.8665  
MCC: 0.6993 

#### Lower 480 Upper 1800
Final Evaluation on Training Set:   
Accuracy: 0.8240    
F1 Score: 0.7447    
Precision: 0.6913   
Recall: 0.8815  
MCC: 0.7157 

### Time Hypothesis Test

In [ ]:
# Final test set evaluation
X_test = satisfy_test_monir.drop(columns=["q61"])
y_test = satisfy_test_monir["q61"]

y_pred = satisfy_mali_model.predict(X_test)

len(y_pred)

print("\nVotingClassifier Evaluation on Test Set:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro', zero_division=1):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro', zero_division=1):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred):.4f}")

#### No Bounds
Final Evaluation on Test Set:   
Accuracy: 0.6324    
F1 Score: 0.5306    
Precision: 0.5033   
Recall: 0.5971  
MCC: 0.4033 

#### Lower 300 Upper 2500 v
Final Evaluation on Test Set:   
Accuracy: 0.6295    
F1 Score: 0.5241    
Precision: 0.4985   
Recall: 0.5878  
MCC: 0.3961 

#### Lower 300 Upper None ^
Final Evaluation on Test Set:   
Accuracy: 0.6297    
F1 Score: 0.5284    
Precision: 0.5010   
Recall: 0.5932  
MCC: 0.4004 

#### Lower 3000 Upper None ^^
Final Evaluation on Test Set:   
Accuracy: 0.6493    
F1 Score: 0.5188    
Precision: 0.5201   
Recall: 0.5208  
MCC: 0.3934

#### Lower 3000 Upper 25000 ^
Final Evaluation on Test Set (satisfy_male_test):   
Accuracy: 0.6352    
F1 Score: 0.5173    
Precision: 0.5077   
Recall: 0.5350  
MCC: 0.3764 

#### Lower 3000 Upper 250000
Final Evaluation on Test Set (satisfy_male_test):   
Accuracy: 0.6523    
F1 Score: 0.4964    
Precision: 0.5083   
Recall: 0.4997  
MCC: 0.3912 

#### Lower 3000 Upper 1M
Final Evaluation on Test Set (satisfy_male_test):   
Accuracy: 0.6510    
F1 Score: 0.5241    
Precision: 0.5165   
Recall: 0.5356  
MCC: 0.4044

## Fulfill

### Time Hypothesis Training

In [ ]:
fulfill_mali_model = Pipeline([
    ("preprocessing", fulfill_pipeline),
    ("Voting", VotingClassifier(estimators=[
        ("hgb", HistGradientBoostingClassifier(
            learning_rate=0.02883701168472723, max_iter=312, max_leaf_nodes=37, min_samples_leaf=15,
            max_depth=7, l2_regularization=0.3398231196727128, early_stopping=True,
            n_iter_no_change=15, random_state=42, class_weight="balanced"
        )),
        ("RandomForest", RandomForestClassifier(
            n_estimators=379,             
            max_depth=18,              
            min_samples_split=5,         
            min_samples_leaf=3,           
            max_features="sqrt",      
            class_weight="balanced",     
            bootstrap=False,              
            random_state=42,
        )),
    ], voting='soft')),
])

In [ ]:
# Final training set evaluation
fulfill_training_monir_trollfree = remove_trolls(fulfill_training_monir, lower = 480, upper=1800)
print(len(fulfill_training_monir_trollfree))
X_train = fulfill_training_monir_trollfree.drop(columns=["q62"])
y_train = fulfill_training_monir_trollfree["q62"]

fulfill_mali_model.fit(X_train, y_train)

y_pred = fulfill_mali_model.predict(X_train)

len(y_pred)

print("\nVotingClassifier Evaluation on Training Set:")
print(f"Accuracy: {accuracy_score(y_train, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_train, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_train, y_pred, average='macro', zero_division=1):.4f}")
print(f"Recall: {recall_score(y_train, y_pred, average='macro', zero_division=1):.4f}")
print(f"MCC: {matthews_corrcoef(y_train, y_pred):.4f}")

11923

VotingClassifier Evaluation on Training Set:
Accuracy: 0.9088
F1 Score: 0.9081
Precision: 0.8799
Recall: 0.9432
MCC: 0.8543


In [ ]:
with open('fulfill_RubberCorn.dill', 'wb') as file:
    dill.dump(fulfill_mali_model, file)

Hard voting:    
Final Evaluation on Training Set:   
Accuracy: 0.8525    
F1 Score: 0.7621    
Precision: 0.7145   
Recall: 0.8963  
MCC: 0.7614 

In [ ]:
# Final test set evaluation
X_test = fulfill_test_monir.drop(columns=["q62"])
y_test = fulfill_test_monir["q62"]

y_pred = fulfill_mali_model.predict(X_test)

len(y_pred)

print("\nFinal Evaluation on Test Set using :")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, average='macro', zero_division=1):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, average='macro', zero_division=1):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred):.4f}")

In [ ]:
scoring_metrics = {
    'accuracy': 'accuracy',
    'mcc': make_scorer(matthews_corrcoef),
    'f1': 'f1_macro',
    'precision': make_scorer(precision_score, zero_division=1, average='macro'),
    'recall': make_scorer(recall_score, zero_division=1, average='macro')
}

cv_results_sat_rf = cross_validate(fulfill_RF_pipeline, fulfill_training_monir_X, fulfill_training_monir_y, cv=4, scoring=scoring_metrics, return_train_score=True)
cv_results_sat_xgb = cross_validate(fulfill_XGB_pipeline, fulfill_training_monir_X, fulfill_target_encoder.fit_transform(fulfill_training_monir_y), cv=4, scoring=scoring_metrics, return_train_score=True)
cv_results_sat_voting = cross_validate(fulfill_mali_model, fulfill_training_monir_X, fulfill_training_monir_y, cv=4, scoring=scoring_metrics, return_train_score=True)

rf_scores = {metric: cv_results_sat_rf[f"test_{metric}"] for metric in scoring_metrics}
xgb_scores = {metric: cv_results_sat_xgb[f"test_{metric}"] for metric in scoring_metrics}
voting_scores = {metric: cv_results_sat_voting[f"test_{metric}"] for metric in scoring_metrics}

In [ ]:
for metric in rf_scores:
    print(f"Metric: {metric}")
    
    f_val, p_val = f_oneway(rf_scores[metric], xgb_scores[metric], voting_scores[metric])
    print(f"{f_val:.4f}, {p_val:.4f}", end=" ")
    if p_val < 0.05:
        print(f"→ Statistically significant difference in *{metric}* between algorithms.")
    else:
        print(f"→ No statistically significant difference in *{metric}* between algorithms.")

In [ ]:
for metric in rf_scores:
    print(f"\nMetric: {metric}")
    print(f"  RF mean:     {np.mean(rf_scores[metric]):.4f}")
    print(f"  XGB mean:    {np.mean(xgb_scores[metric]):.4f}")
    print(f"  Voting mean: {np.mean(voting_scores[metric]):.4f}")

In [ ]:
def run_tukey_tests(cv_scores, dataset_name, export_csv=False):
    tukey_results = {}

    print(f"\nTukey HSD Tests for {dataset_name}")

    for metric_name, model_scores in cv_scores.items():
        print(f"\nMetric: {metric_name}")
        
        model_names = list(model_scores.keys())
        score_arrays = [model_scores[model] for model in model_names]
        result = tukey_hsd(*score_arrays)
        tukey_results[metric_name] = result
        comparisons = []
        for ((i, j), stat) in np.ndenumerate(result.statistic):
            if i < j:
                comparisons.append({
                    "Model 1": model_names[i],
                    "Model 2": model_names[j],
                    "Statistic": stat,
                    "p-value": result.pvalue[i, j],
                    "CI Lower": result.confidence_interval().low[i, j],
                    "CI Upper": result.confidence_interval().high[i, j],
                    "Significant": result.pvalue[i, j] < 0.05
                })

        df = pd.DataFrame(comparisons)
        df["Significant"] = df["Significant"].map({True: "yes", False: "no"})
        print(df.to_string(index=False))

        if export_csv:
            df.to_csv(f"tukey_{dataset_name.lower().replace(' ', '_')}_{metric_name}.csv", index=False)

    return tukey_results

In [ ]:
cv_scores = {}

for metric in rf_scores:
    cv_scores[metric] = {
        "RF": rf_scores[metric],
        "XGB": xgb_scores[metric],
        "Voting": voting_scores[metric]
    }

In [ ]:
cv_scores

In [ ]:
tukey_results = run_tukey_tests(cv_scores, "Fulfill RubberCorn")